## Data Prep Mortality rate, Human Development Index and Regions/Super_Regions

In [1]:
import numpy as np
import pandas as pd

from os import path

In [2]:
p = path.join('Data', 'IHME-GBD_2016_Lung.csv')
p = path.abspath(p)

In [3]:
# Data are collected from: Global Health Data Exchange
LC_url = "Data/IHME-GBD_2016_Lung.csv"
LC_df = pd.read_csv(LC_url)

#Data are collected from: UNITED NATIONS DEVELOPMENT PROGRAMME, Human Development Reports

HDI_url = "Data/Human development index (HDI)1990-2015.csv"
HDI_df = pd.read_csv(HDI_url)


r_sr_url = "Data/r_rs.csv"
r_sr_df = pd.read_csv(r_sr_url)
r_sr_df.drop("Unnamed: 0", axis=1,inplace=True)

In [4]:
LC_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15795 entries, 0 to 15794
Data columns (total 10 columns):
measure     15795 non-null object
location    15795 non-null object
sex         15795 non-null object
age         15795 non-null object
cause       15795 non-null object
metric      15795 non-null object
year        15795 non-null int64
val         15795 non-null float64
upper       15795 non-null float64
lower       15795 non-null float64
dtypes: float64(3), int64(1), object(6)
memory usage: 1.2+ MB


> ### Need to rename year columns to be able to change the current wide form into long form   

In [5]:
for i in range(2,HDI_df.columns[2:].size+2):
    HDI_df.rename(columns={HDI_df.columns[i] : "HDI-"+str(HDI_df.columns[i])}, inplace=True)

In [6]:
HDI_df_long = pd.wide_to_long(HDI_df,"HDI", i= "Country" ,j = "Year", sep="-", suffix='\w')
HDI_df_long.sort_index(inplace=True)
HDI_df_long.reset_index(inplace=True)

> ### Need to add a column for  HDI from long form dataframe HDI_df_long

In [7]:
LC_df["HDI"] = ""
for i in range(LC_df.shape[0]):
    for j in range(HDI_df.shape[0]) :
        if LC_df.iloc[i,1]==HDI_df_long.iloc[j,0].strip()and str(LC_df.iloc[i,6]) ==HDI_df_long.iloc[j,1]: 
            LC_df.iloc[i,10] = float(HDI_df_long.iloc[j,3])


> ### Need to add region and super region 

In [8]:
LC_df["region"] =""
LC_df["super_region"]=""

In [9]:
for i in range(LC_df.shape[0]):
    for j in range(r_sr_df.shape[0]):
        if LC_df.iloc[i,1] == r_sr_df.iloc[j,0]:
            LC_df.iloc[i,11] = r_sr_df.iloc[j,1]
            LC_df.iloc[i,12] = r_sr_df.iloc[j,2]

In [10]:
LC_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15795 entries, 0 to 15794
Data columns (total 13 columns):
measure         15795 non-null object
location        15795 non-null object
sex             15795 non-null object
age             15795 non-null object
cause           15795 non-null object
metric          15795 non-null object
year            15795 non-null int64
val             15795 non-null float64
upper           15795 non-null float64
lower           15795 non-null float64
HDI             15663 non-null object
region          15795 non-null object
super_region    15795 non-null object
dtypes: float64(3), int64(1), object(9)
memory usage: 1.6+ MB


In [11]:
LC_df.to_csv("Data/LC_HDI.csv")